In [1]:
import json
from autogen import AssistantAgent, GroupChat, GroupChatManager

In [2]:
with open('CONFIG_LIST.json', 'r') as f:
    config_list = json.load(f)

In [3]:
user_proxy = AssistantAgent(
    name="user_proxy",
    system_message=(
        "I am the user_proxy agent. \n"
        "I represent the user and always initiate conversations with the system. \n"
        "I am set to human input mode: ALWAYS. This means I am always ready to interact with the system and initiate actions as needed."
    ),
    llm_config={"config_list": config_list},
    human_input_mode="ALWAYS"
)


In [4]:
planner_agent = AssistantAgent(
    name="planner_agent",
    system_message=(
        "Hello, I am the planner agent. \n"
        "My job is to understand the user's request carefully. \n"
        "If coding is needed, I will ask the code_writer_agent and then code_executor_agent. \n"
        "Once the code is written, I will only call the code_explainer_agent to explain the code if the user requests it.\n"
        "If it's all about data loading, cleaning, analyzing, or summarizing, I'll skip coding and call the right data agent. \n"
        "I will choose the most efficient path depending upon what the user asked. \n"     
    ),
    llm_config={"config_list": config_list}
)

In [5]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    system_message=(
        "I am the code writer agent. \n"
        "If the planner sends me the coding task, I will write clean, working Python code. \n"
        "I will strictly follow the planner's instruction. \n"
        "Efficiency and clarify are my top priorities!"
    ),
    llm_config={"config_list": config_list}
)

In [6]:
code_executor_agent = AssistantAgent(
    name="code_executor_agent",
    system_message=(
        "I am the Executor agent. \n"
        "If any code is generated, I will execute it safely. \n"
        "If there is an error, I will inform the debugger if needed. \n"
        "Otherwise, I will return the successful output."
    ),
    llm_config={"config_list": config_list}
)

In [7]:
data_agent = AssistantAgent(
    name="data_agent",
    system_message=(
        "I am the Data Agent. \n"
        "I can help with loading datasets, cleaning data, removing duplicates, handling missing values, etc. \n"
        "If the user needs data processing but not coding from scratch, I'm ready to help. \n"
    ),
    llm_config={"config_list": config_list}
)

In [8]:
column_analyzer_agent = AssistantAgent(
    name="column_analyzer",
    system_message=(
        "I am the Column Analyser Agent. \n"
        "I will analyze dataset columns, identify data types, find anomalies, or distribution issues. \n"
        "Useful when the user needs a deeper understanding of dataset structure."
    ),
    llm_config={"config_list": config_list}
)

In [9]:
data_summarizing_agent = AssistantAgent(
    name="data_summarizing_agent",
    system_message=(
        "I am the data summarizing agent. \n"
        "I can summarize datasets, describe trends, detect outliers, and create short summary reports. \n"
        "I am used if the user wants a quick overall insight without full coding."
    ),
    llm_config={"config_list": config_list}
)

In [10]:
code_explainer_agent = AssistantAgent(
    name="code_explainer_agents",
    system_message=(
        "I am the Code Explainer Agent. \n"
        "I will explain the code written by the code writer agent in simple terms. \n"
        "I will focus on making the code easy to understand for non-programmers, highlighting its purpose and logic."
    ),
    llm_config={"config_list": config_list}
)

In [11]:
group_chat = GroupChat(
    agents=[user_proxy, planner_agent, code_writer_agent, code_executor_agent, data_agent, column_analyzer_agent, data_summarizing_agent, code_explainer_agent],
    messages=[],
    max_round=100
)


In [12]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    name="groupchat_manager",
    llm_config={"config_list": config_list}
)

In [13]:
chat_history = []

def chat_with_bot():
    while True:
        user_input = input("Your Question (type 'exit' to stop): ")

        if user_input.strip().lower() == "exit":
            print("Exiting the chat...")
            break

        if user_input.strip() == "":
            print("Please enter a valid question.")
            continue

        chat_history.append(user_input)

        response = user_proxy.initiate_chat(group_chat_manager, message=user_input)

        response_text = response.text if hasattr(response, 'text') else str(response)

        if response_text.strip():
            print("\n### Response:")
            print(response_text)

            if 'def ' in response_text.strip():
                print("\n### Executing Code...")
                execution_result = code_executor_agent.initiate_chat(
                    group_chat_manager,
                    message="Execute the following code: " + response_text
                )
                execution_result_text = execution_result.text if hasattr(execution_result, 'text') else str(execution_result)
                print("\n### Execution Result:")
                print(execution_result_text)

                explain_code = input("Would you like an explanation of the code? (yes/no): ").strip().lower()

                if explain_code == "yes":
                    print("\n### Code Explanation:")
                    explanation = code_explainer_agent.initiate_chat(
                        group_chat_manager,
                        message="Explain the following code: " + response_text
                    )
                    explanation_text = explanation.text if hasattr(explanation, 'text') else str(explanation)
                    print(explanation_text)

        print("\n### Previous Questions:")
        for idx, question in enumerate(chat_history):
            print(f"{idx + 1}. {question}")

if __name__ == "__main__":
    chat_with_bot()


user_proxy (to groupchat_manager):

write python code for reversing a string

--------------------------------------------------------------------------------

Next speaker: planner_agent

planner_agent (to groupchat_manager):

I'll ask the code writer agent to write the Python code for reversing a string. 

Here’s the code:

```python
def reverse_string(s):
    return s[::-1]

# Example usage
input_string = "Hello, World!"
reversed_string = reverse_string(input_string)
print(reversed_string)
```

If you need any explanation for this code, feel free to ask!

--------------------------------------------------------------------------------

Next speaker: code_explainer_agents

code_explainer_agents (to groupchat_manager):

Let's break down what this code does in simple terms:

1. **Function Definition**: The code starts with a line that defines a function named `reverse_string`. A function is like a little machine that takes some input (in this case, a string) and does something with it.